This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [30]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [31]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix)
GRAPH_NAME = "frbny_sce_analysis"

# Set the path to the graph output folder
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

3. Set the fed_2025 template as default

In [32]:
pio.templates.default = 'fed_2025'



In [33]:
GRAPH_OUTPUT_PATH

WindowsPath('C:/Users/kaika/NEUFed2025/figures')

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [73]:

excel_file = pd.ExcelFile("FRBNY-SCE-Data (1).xlsx")
sheet_names = excel_file.sheet_names
print("Available sheets:", sheet_names)


df = pd.read_excel("FRBNY-SCE-Data (1).xlsx", sheet_name='Inflation expectations', skiprows=2)

df.columns = df.iloc[0]
df = df[1:]
df = df.reset_index(drop=True)


Available sheets: ['Disclaimer', 'License', 'Notes', 'Inflation expectations', 'Inflation expectations Demo', 'Inflation uncertainty', 'Inflation uncertainty Demo', 'Home price expectations', 'Home price expectations Demo', 'Home price uncertainty', 'Home price uncertainty Demo', 'Commodity expectations', 'Earnings growth', 'Earnings growth Demo', 'Earnings uncertainty', 'Earnings uncertainty Demo', 'Job separation expectation', 'Job separation expectation Demo', 'Job finding expectations', 'Job finding expectations Demo', 'Moving expectations', 'Moving expectations Demo', 'Unemployment Expectations', 'Unemployment Expectations Demo', 'HH Income Change', 'HH Income Change Demo', 'HH Spending Change', 'HH Spending Change Demo', 'Taxes Change', 'Taxes Change Demo', 'Credit availability', 'Household financial situation', 'Delinquency expectations', 'Delinquency expectations Demo', 'Interest rate expectations', 'Interest rate expectations Demo', 'Stock Prices', 'Stock Prices Demo', 'Govern

In [74]:
df.columns = df.columns.str.strip()

df = df[df.iloc[:, 0].notna()]

date_column = df.columns[0]

df = df[pd.to_numeric(df[date_column], errors='coerce').notna()]
df[date_column] = pd.to_datetime(df[date_column], format='%Y%m')
df.set_index(date_column, inplace=True)
df.index.name = "Date"

df = df.apply(pd.to_numeric, errors='coerce')

df = df[['Median one-year ahead expected inflation rate', 'Median three-year ahead expected inflation rate']]

print("\nCleaned data shape:", df.shape)
df.tail()


Cleaned data shape: (148, 2)


,Median one-year ahead expected inflation rate,Median three-year ahead expected inflation rate
Date,,
2025-05-01,3.196597,3.000000
2025-06-01,3.015459,2.997374
2025-07-01,3.090884,3.000000
2025-08-01,3.196597,3.000000
2025-09-01,3.377147,3.046669


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [75]:
fig = go.Figure()

for col in df.columns:
    fig.add_trace(
        go.Scatter(
            x=df.index,
            y=df[col],
            mode='lines',
            name=col
        )
    )

fig.update_layout(
    title = dict(text = 'FRBNY Survey of Consumer Inflation Expectations '),
    xaxis_title="Date",
    yaxis_title="Percent",
)

fig.update_yaxes(tickformat=".2f")

fig.update_xaxes(
    type='date',
    tickformat='%Y',
)

ut.add_end_labels(fig = fig)

fig.show()

fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")